In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler





# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.tail(1)

data = data.iloc[:10747]

data.tail(1)


data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()



#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)

data["close"] = data["close"].astype(float)

print(data.tail(1))

             date    open    high     low   close      volume  adjusted_close  \
10746  2023-07-31  196.06  196.49  195.26  196.45  38824100.0        196.1851   

       change_percent  avg_vol_20d  
10746            0.32   49803320.0  


Transform: date to torch: date_embeddings

In [2]:



import torch
import torch.nn as nn
import pandas as pd

# Sample data with a 'date' column containing dates from 1980-12-12 to 2023-07-31
data2 = pd.DataFrame({'date': pd.date_range(start='1980-12-12', end='2023-07-31')})

# Convert 'date' column to datetime
data2['date'] = pd.to_datetime(data['date'])

# Extract day, month, and year from the date column
data['day'] = data2['date'].dt.day
data['month'] = data2['date'].dt.month
data['year'] = data2['date'].dt.year

# Define the embedding dimensions
embedding_dim = 1  # You can adjust this dimension as needed

# Create embedding layers for day, month, and year
day_embedding = nn.Embedding(32, embedding_dim)  # 0-31 days
month_embedding = nn.Embedding(13, embedding_dim)  # 1-12 months
year_embedding = nn.Embedding(44, embedding_dim)  # Embedding for years from 1980 to 2023

# Convert day, month, and year to tensors with Long data type
day_tensor = torch.LongTensor(data['day'].values)
month_tensor = torch.LongTensor(data['month'].values)
year_tensor = torch.LongTensor(data['year'].values - 1980)  # Convert years to an index from 0 to 43


# Pass tensors through embedding layers to get embeddings
day_embeddings = day_embedding(day_tensor)
month_embeddings = month_embedding(month_tensor)
year_embeddings = year_embedding(year_tensor)

# Concatenate the embeddings
date_embeddings = torch.cat((day_embeddings, month_embeddings, year_embeddings), dim=1)

# Print the resulting embeddings



Transform feature to tensor 
Concatenate date embeddings with data
Split the data into training and testing sets

In [3]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Convert data to numpy arrays
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']].values
y_data = data["close"].values.reshape(-1, 1)  # Reshape y_data if it's 1D

# Concatenate date embeddings with data
x_feature_tensors = torch.tensor(x_data, dtype=torch.float32)
x_combined = torch.cat((x_feature_tensors, date_embeddings), dim=1)

y_feature_tensors = torch.tensor(y_data, dtype=torch.float32)
y_combined = torch.cat((y_feature_tensors, date_embeddings), dim=1)

# Split the data into training and testing sets
train_size = int(len(x_combined) * 0.67)
x_train, x_test = x_combined[:train_size], x_combined[train_size:]
y_train, y_test = y_combined[:train_size], y_combined[train_size:]

# Initialize the scaler
scaler = MinMaxScaler()



# Fit the scaler only on the training data and transform both training and test data
x_train_scaled = scaler.fit_transform(x_train.detach().numpy())
y_train_scaled = scaler.fit_transform(y_train.detach().numpy())



# fitting the MinMaxScaler to your training data using fit_transform and then
# applying the same transformation (based on the statistics learned) to your test data using transform
x_train_scaled = scaler.fit_transform(x_train.detach().numpy())
x_test_scaled = scaler.transform(x_test.detach().numpy())  # Use transform, not fit_transform

y_train_scaled = scaler.fit_transform(y_train.detach().numpy())
y_test_scaled = scaler.transform(y_test.detach().numpy())  # Use transform, not fit_transform

# Convert back to tensors if needed
x_train_scaled = torch.tensor(x_train_scaled, dtype=torch.float32)
x_test_scaled = torch.tensor(x_test_scaled, dtype=torch.float32)

y_train_scaled = torch.tensor(y_train_scaled, dtype=torch.float32)
y_test_scaled = torch.tensor(y_test_scaled, dtype=torch.float32)




LSTM Model 
Initialize the model, loss, and optimizer
Training using Walk-Forward Validation
Evaluate the model on the test set

In [4]:
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np

# Adjustable Parameters
learning_rate = 0.001
window_size = 50
test_window_size = 50
hidden_dim = 64
n_layers = 2
batch_evaluation_frequency = 10
epochs = len(x_train_scaled) - window_size
batch_size = 1
input_size = x_train_scaled.shape[1]  # Input size based on your dataset
output_size = 1  # Output size (for regression task)

# LSTM Model Definition
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden

# Initialize the model, loss, and optimizer
model = LSTMModel(input_size, hidden_dim, n_layers, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training using Walk-Forward Validation
for i in range(window_size, epochs):
    optimizer.zero_grad()

    # Extract training window with batch size
    start_idx = i - window_size
    end_idx = i
    x_window = torch.tensor(x_train_scaled[start_idx:end_idx], dtype=torch.float32)
    y_window = torch.tensor(y_train_scaled[start_idx:end_idx], dtype=torch.float32)

    # Reshape x_window to match LSTM input shape [batch_size=1, sequence_length=window_size, input_size=input_size]
    x_window = x_window.view(1, window_size, input_size)

    # Initialize hidden state
    hidden = model.init_hidden(batch_size)

    # Forward pass
    outputs, hidden = model(x_window, hidden)
    loss = criterion(outputs, y_window)

    # Backward pass and optimize
    loss.backward()
    optimizer.step()

    # Evaluate the model on the test set
    if i % batch_evaluation_frequency == 0:
        with torch.no_grad():
            x_test_window = torch.tensor(x_test_scaled[:test_window_size], dtype=torch.float32)
            y_test_window = torch.tensor(y_test_scaled[:test_window_size], dtype=torch.float32)

            # Reshape x_test_window to match LSTM input shape [batch_size=1, sequence_length=test_window_size, input_size=input_size]
            x_test_window = x_test_window.view(1, test_window_size, input_size)

            # Initialize hidden state for test
            hidden = model.init_hidden(batch_size)

            test_outputs, _ = model(x_test_window, hidden)
            test_loss = criterion(test_outputs, y_test_window)
            print(f"Iteration {i}, Train Loss: {loss.item()}, Test Loss: {test_loss.item()}")


C:\Users\pitr7\AppData\Local\Temp\ipykernel_6112\1959663930.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_window = torch.tensor(x_train_scaled[start_idx:end_idx], dtype=torch.float32)
C:\Users\pitr7\AppData\Local\Temp\ipykernel_6112\1959663930.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_window = torch.tensor(y_train_scaled[start_idx:end_idx], dtype=torch.float32)


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([50, 4])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\pitr7\AppData\Local\Temp\ipykernel_6112\1959663930.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_window = torch.tensor(x_test_scaled[:test_window_size], dtype=torch.float32)
C:\Users\pitr7\AppData\Local\Temp\ipykernel_6112\1959663930.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test_window = torch.

Iteration 50, Train Loss: 0.19831621646881104, Test Loss: 0.2544884979724884
Iteration 60, Train Loss: 0.11528627574443817, Test Loss: 0.10409238934516907
Iteration 70, Train Loss: 0.10092315822839737, Test Loss: 0.13713563978672028
Iteration 80, Train Loss: 0.08645936101675034, Test Loss: 0.13324856758117676
Iteration 90, Train Loss: 0.07585611939430237, Test Loss: 0.13573899865150452
Iteration 100, Train Loss: 0.07785789668560028, Test Loss: 0.15177147090435028
Iteration 110, Train Loss: 0.0873619094491005, Test Loss: 0.14349226653575897
Iteration 120, Train Loss: 0.09891732037067413, Test Loss: 0.13104158639907837
Iteration 130, Train Loss: 0.10746145248413086, Test Loss: 0.1289627104997635
Iteration 140, Train Loss: 0.12318892776966095, Test Loss: 0.11817094683647156
Iteration 150, Train Loss: 0.13573236763477325, Test Loss: 0.11377139389514923
Iteration 160, Train Loss: 0.14070811867713928, Test Loss: 0.11447585374116898
Iteration 170, Train Loss: 0.1264667958021164, Test Loss: 0.

In [5]:
# Number of future time steps you want to predict
num_future_steps = 1

# Set the model to evaluation mode
model.eval()

# Initialize hidden state
hidden = model.init_hidden(1)  # Batch size is 1

# Initialize the input sequence with the last window_size data points from the training set
input_sequence = x_train_scaled[-window_size:].tolist()

future_predictions = []

with torch.no_grad():
    for _ in range(num_future_steps):
        # Convert the input_sequence to a tensor and reshape
        input_tensor = torch.tensor(input_sequence, dtype=torch.float32).view(1, window_size, input_size)

        # Get prediction from model
        prediction, hidden = model(input_tensor, hidden)



In [6]:
import torch
import torch.nn as nn
import pandas as pd

# Sample data with a 'date' column containing dates from 1980-12-12 to 2023-07-31
data2 = pd.DataFrame({'date': pd.date_range(start='1980-12-12', end='2023-07-31')})

# Convert 'date' column to datetime
data2['date'] = pd.to_datetime(data2['date'])

# Extract day, month, and year from the date column for August 1st, 2023
day_august_1 = 1  # Day is 1st
month_august_1 = 8  # Month is August
year_august_1 = 2023  # Year is 2023

# Define the embedding dimensions
embedding_dim = 1

# Create embedding layers for day, month, and year
day_embedding = nn.Embedding(32, embedding_dim)  # 0-31 days
month_embedding = nn.Embedding(13, embedding_dim)  # 1-12 months
year_embedding = nn.Embedding(44, embedding_dim)  # Embedding for years from 1980 to 2023

# Convert day, month, and year to tensors with Long data type
day_tensor = torch.LongTensor([day_august_1])
month_tensor = torch.LongTensor([month_august_1])
year_tensor = torch.LongTensor([year_august_1 - 1980])  # Convert years to an index from 0 to 43

# Pass tensors through embedding layers to get embeddings
day_embeddings_august_1 = day_embedding(day_tensor)
month_embeddings_august_1 = month_embedding(month_tensor)
year_embeddings_august_1 = year_embedding(year_tensor)

# Concatenate the embeddings for August 1st, 2023
date_embeddings_august_1 = torch.cat((day_embeddings_august_1, month_embeddings_august_1, year_embeddings_august_1), dim=1)

# Print the embeddings for August 1st, 2023
print("Embeddings for August 1st, 2023:")
print(date_embeddings_august_1)


Embeddings for August 1st, 2023:
tensor([[-0.3946,  2.2370,  0.8878]], grad_fn=<CatBackward0>)


In [11]:
"""
import torch
import torch.nn as nn
import pandas as pd

# Sample data with a 'date' column containing dates from 1980-12-12 to 2023-07-31
data2 = pd.DataFrame({'date': pd.date_range(start='1980-12-12', end='2023-07-31')})

# Convert 'date' column to datetime
data2['date'] = pd.to_datetime(data2['date'])

# Extract day, month, and year from the date column for August 1st
day_august_1 = data2['date'].dt.day.iloc[0]
month_august_1 = data2['date'].dt.month.iloc[0]
year_august_1 = data2['date'].dt.year.iloc[0]

# Define the embedding dimensions
embedding_dim = 1

# Create embedding layers for day, month, and year
day_embedding = nn.Embedding(32, embedding_dim)  # 0-31 days
month_embedding = nn.Embedding(13, embedding_dim)  # 1-12 months
year_embedding = nn.Embedding(44, embedding_dim)  # Embedding for years from 1980 to 2023

# Convert day, month, and year to tensors with Long data type
day_tensor = torch.LongTensor([day_august_1])
month_tensor = torch.LongTensor([month_august_1])
year_tensor = torch.LongTensor([year_august_1 - 1980])  # Convert years to an index from 0 to 43

# Pass tensors through embedding layers to get embeddings
day_embeddings_august_1 = day_embedding(day_tensor)
month_embeddings_august_1 = month_embedding(month_tensor)
year_embeddings_august_1 = year_embedding(year_tensor)

# Concatenate the embeddings for August 1st
date_embeddings_august_1 = torch.cat((day_embeddings_august_1, month_embeddings_august_1, year_embeddings_august_1), dim=1)
"""

In [12]:
# Print or use y_predicted and date_embeddings_combined as needed
print("y_predicted:", y_predicted[0].item())  # This will print the single scalar value of y_predicted
print("Date Embeddings Combined:", date_embeddings_combined)


NameError: name 'y_predicted' is not defined

In [ ]:
forecasted_value

tensor([[0.4098]])

In [ ]:
print(y_train_scaled.shape)

torch.Size([7200, 4])


In [ ]:
# Forecast the next 'num_forecast_steps' values
for _ in range(num_forecast_steps):
    with torch.no_grad():
        # Predict the next value by passing both input_sequence and hidden state
        forecasted_value, _ = model(input_sequence, hidden)

# Convert forecasted_value tensor to numpy array and reshape to (1, 1)
forecasted_value_numpy = forecasted_value.cpu().numpy().reshape(1, 1)

# Inverse transform the forecasted value
forecasted_value_original = scaler.inverse_transform(forecasted_value_numpy)

# Print or use the original forecasted value
print("Original Forecasted Value:", forecasted_value_original)


AttributeError: 'list' object has no attribute 'size'

In [13]:
# Number of future time steps you want to predict
num_future_steps = 1

# Set the model to evaluation mode
model.eval()

# Initialize hidden state
hidden = model.init_hidden(1)  # Batch size is 1

# Initialize the input sequence with the last window_size data points from the training set
input_sequence = x_train_scaled[-window_size:].tolist()

future_predictions = []

with torch.no_grad():
    for _ in range(num_future_steps):
        # Convert the input_sequence to a tensor and reshape
        input_tensor = torch.tensor(input_sequence, dtype=torch.float32).view(1, window_size, input_size)

        # Get prediction from model
        prediction, hidden = model(input_tensor, hidden)

        # Append the prediction to future_predictions
        future_predictions.append(prediction.item())

        # Update input_sequence with the new prediction (for the next iteration)
        input_sequence.append(prediction.item())
        input_sequence = input_sequence[-window_size:]  # Keep only the last window_size elements

# Inverse transform the scaled predictions to get the actual predicted "close" values
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Print or use future_predictions as needed
print(future_predictions)


ValueError: non-broadcastable output operand with shape (1,1) doesn't match the broadcast shape (1,4)

In [ ]:
print(input_tensor.shape)

torch.Size([1, 50, 10])


In [ ]:
# Number of future time steps you want to predict
num_future_steps = 10

# Set the model to evaluation mode
model.eval()

# Initialize hidden state
hidden = model.init_hidden(1)  # Batch size is 1

# Initialize the input sequence with the last window_size data points from the training set
input_sequence = x_train_scaled[-window_size:].tolist()

future_predictions = []

with torch.no_grad():
    for _ in range(num_future_steps):
        # Convert the input_sequence to a tensor and reshape
        input_tensor = torch.tensor([input_sequence], dtype=torch.float32)
        input_tensor = input_tensor.view(1, window_size, input_size)

        # Get prediction from model
        prediction, hidden = model(input_tensor, hidden)

        # Append the prediction to future_predictions
        future_predictions.append(prediction.item())

        # Update input_sequence with the new prediction (for the next iteration)
        # Note: For simplicity, we're updating only one feature, assuming it's the 'close' value.
        # If you have multiple features in the input_sequence, adjust accordingly.
        input_sequence.append(prediction.item())
        input_sequence = input_sequence[-window_size:]  # Keep only the last window_size elements

# Inverse transform the scaled predictions to get the actual predicted "close" values
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Print or use future_predictions as needed
print(future_predictions)


TypeError: not a sequence

In [ ]:
# Number of future time steps you want to predict
num_future_steps = 10  # For example, predict the next 10 days

# Use the last window_size data points from the training set to start the prediction
input_sequence = x_train_scaled[-window_size:].tolist()  # Convert tensor to list

# Set the model to evaluation mode
model.eval()

# Predict future values
future_predictions = []

# Initialize hidden state
hidden = model.init_hidden(1)  # Batch size is 1

with torch.no_grad():
    for _ in range(num_future_steps):
        # Convert input_sequence to tensor and reshape
        input_tensor = torch.tensor(input_sequence[-window_size:], dtype=torch.float32).view(1, window_size, input_size)

        # Get prediction from model
        prediction, hidden = model(input_tensor, hidden)

        # Append prediction to future_predictions
        future_predictions.append(prediction.item())

        # Update input_sequence with new prediction (for next iteration)
        input_sequence.append(prediction.item())

# Inverse transform the scaled predictions to get actual predictions
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Print or use future_predictions as needed
print(future_predictions)


TypeError: not a sequence

In [ ]:
print(input_tensor.shape)

torch.Size([1, 50, 10])


In [ ]:
# Set the model to evaluation mode
model.eval()

# Define the number of future steps you want to forecast
num_forecast_steps = 1

# Initialize the input sequence for forecasting
input_sequence = x_train_scaled[-window_size:].view(1, window_size, input_size)

# Initialize hidden state for forecasting
hidden = model.init_hidden(1)  # Assuming batch size is 1

# Forecast the next 'num_forecast_steps' values
for _ in range(num_forecast_steps):
    with torch.no_grad():
        # Predict the next value by passing both input_sequence and hidden state
        forecasted_value, _ = model(input_sequence, hidden)

        # Print the forecasted value directly without reshaping
        print("Forecasted value tensor:", forecasted_value)

        # Update the input sequence by removing the first element along the sequence dimension
        # and adding the forecasted value at the end
        input_sequence = torch.cat((input_sequence[:, 1:, :], forecasted_value), dim=1)

# Convert the forecasted value back to the original scale using the scaler
forecasted_value_unscaled = scaler.inverse_transform(forecasted_value.cpu().numpy())

print(f"Forecasted value of 'close' for the next time step: {forecasted_value_unscaled[0][0][0]}")


Forecasted value tensor: tensor([[0.4852]])


RuntimeError: Tensors must have same number of dimensions: got 3 and 2

In [ ]:
# Set the model to evaluation mode
model.eval()

# Define the number of future steps you want to forecast
num_forecast_steps = 1

# Initialize the input sequence for forecasting
input_sequence = x_train_scaled[-window_size:].view(1, window_size, input_size)

# Initialize hidden state for forecasting
hidden = model.init_hidden(1)  # Assuming batch size is 1

# Forecast the next 'num_forecast_steps' values
for _ in range(num_forecast_steps):
    with torch.no_grad():
        # Predict the next value by passing both input_sequence and hidden state
        forecasted_value, _ = model(input_sequence, hidden)

        # Ensure that you get the correct size for the forecasted_value
        forecasted_value_size = forecasted_value.size(-1)  # Access the last dimension

        # Reshape the forecasted value to match the size of the input_sequence
        forecasted_value = forecasted_value.view(1, 1, input_size)

        # Update the input sequence by removing the first element along the sequence dimension
        # and adding the forecasted value at the end
        input_sequence = torch.cat((input_sequence[:, 1:, :], forecasted_value), dim=1)

# Convert the forecasted value back to the original scale using the scaler
forecasted_value_unscaled = scaler.inverse_transform(forecasted_value.cpu().numpy())

print(f"Forecasted value of 'close' for the next time step: {forecasted_value_unscaled[0][0][0]}")


RuntimeError: shape '[1, 1, 10]' is invalid for input of size 1

In [ ]:
# Forecast the next 'num_forecast_steps' values
for _ in range(num_forecast_steps):
    with torch.no_grad():
        # Predict the next value by passing both input_sequence and hidden state
        forecasted_value, _ = model(input_sequence, hidden)

        # Print the shape of forecasted_value
        print("Shape of forecasted_value:", forecasted_value.shape)

        # Update the input sequence by removing the first element along the sequence dimension
        # and adding the forecasted value at the end
        input_sequence = torch.cat((input_sequence[:, 1:, :], forecasted_value), dim=1)


Shape of forecasted_value: torch.Size([1, 1])


RuntimeError: Tensors must have same number of dimensions: got 3 and 2

In [ ]:
# Set the model to evaluation mode
model.eval()

# Define the number of future steps you want to forecast
num_forecast_steps = 1

# Initialize the input sequence for forecasting
input_sequence = x_train_scaled[-window_size:].view(1, window_size, input_size)

# Initialize hidden state for forecasting
hidden = model.init_hidden(1)  # Assuming batch size is 1

# Forecast the next 'num_forecast_steps' values
for _ in range(num_forecast_steps):
    with torch.no_grad():
        # Predict the next value by passing both input_sequence and hidden state
        forecasted_value, _ = model(input_sequence, hidden)

        # Ensure that you get the correct size for the forecasted_value
        forecasted_value_size = forecasted_value.size(-1)  # Access the last dimension

        # Reshape the forecasted value based on its size
        forecasted_value = forecasted_value.view(1, 1, forecasted_value_size)

        # Update the input sequence by removing the first element along the sequence dimension
        # and adding the forecasted value at the end
        input_sequence = torch.cat((input_sequence[:, 1:, :], forecasted_value), dim=1)

# Convert the forecasted value back to the original scale using the scaler
forecasted_value_unscaled = scaler.inverse_transform(forecasted_value.cpu().numpy())

print(f"Forecasted value of 'close' for the next time step: {forecasted_value_unscaled[0][0][0]}")


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 10 but got size 1 for tensor number 1 in the list.

In [ ]:
# Set the model to evaluation mode
model.eval()

# Define the number of future steps you want to forecast
num_forecast_steps = 1

# Initialize the input sequence for forecasting
input_sequence = x_train_scaled[-window_size:].view(1, window_size, input_size)

# Initialize hidden state for forecasting
hidden = model.init_hidden(1)  # Assuming batch size is 1

# Forecast the next 'num_forecast_steps' values
for _ in range(num_forecast_steps):
    with torch.no_grad():
        # Predict the next value by passing both input_sequence and hidden state
        forecasted_value, _ = model(input_sequence, hidden)

        # Ensure that you get the correct size for the forecasted_value
        forecasted_value_size = forecasted_value.size(-1)  # Access the last dimension

        # Reshape the forecasted value based on its size
        forecasted_value = forecasted_value.view(1, 1, forecasted_value_size)

        # Update the input sequence by removing the first element and adding the forecasted value at the end
        input_sequence = torch.cat((input_sequence[:, 1:, :], forecasted_value), dim=1)

# Convert the forecasted value back to the original scale using the scaler
forecasted_value_unscaled = scaler.inverse_transform(forecasted_value.cpu().numpy())

print(f"Forecasted value of 'close' for the next time step: {forecasted_value_unscaled[0][0][0]}")


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 10 but got size 1 for tensor number 1 in the list.

In [ ]:
print("Shape of input_sequence:", input_sequence.shape)
print("Shape of forecasted_value:", forecasted_value.shape)


Shape of input_sequence: torch.Size([1, 50, 10])
Shape of forecasted_value: torch.Size([1, 1, 1])


In [ ]:
# Set the model to evaluation mode
model.eval()

# Define the number of future steps you want to forecast
num_forecast_steps = 1

# Initialize the input sequence for forecasting
input_sequence = x_train_scaled[-window_size:].view(1, window_size, input_size)

# Initialize hidden state for forecasting
hidden = model.init_hidden(batch_size)

# Forecast the next 'num_forecast_steps' values
for _ in range(num_forecast_steps):
    with torch.no_grad():
        # Predict the next value by passing both input_sequence and hidden state
        forecasted_value, _ = model(input_sequence, hidden)

        # Reshape the forecasted value to have the same sequence length as input_sequence
        forecasted_value = forecasted_value.view(1, 1, input_size)

        # Update the input sequence by removing the first element and adding the forecasted value at the end
        input_sequence = torch.cat((input_sequence[:, 1:, :], forecasted_value), dim=1)

# Convert the forecasted value back to the original scale using the scaler
forecasted_value_unscaled = scaler.inverse_transform(forecasted_value.cpu().numpy())

print(f"Forecasted value of 'close' for the next time step: {forecasted_value_unscaled[0][0][0]}")


RuntimeError: shape '[1, 1, 10]' is invalid for input of size 1

In [ ]:
print(f"Shape of input_sequence: {input_sequence.shape}")
print(f"Shape of hidden: {hidden.shape}")




Shape of input_sequence: torch.Size([1, 50, 10])


AttributeError: 'tuple' object has no attribute 'shape'

In [ ]:
# Set the model to evaluation mode
model.eval()

# Define the number of future steps you want to forecast
num_forecast_steps = 1

# Initialize the input sequence for forecasting
input_sequence = x_train_scaled[-window_size:].view(1, window_size, input_size)

# Initialize hidden state for forecasting
hidden = model.init_hidden(1)  # Assuming batch size is 1

# Forecast the next 'num_forecast_steps' values
for _ in range(num_forecast_steps):
    with torch.no_grad():
        # Predict the next value by passing both input_sequence and hidden state
        forecasted_value, _ = model(input_sequence, hidden)

        # Ensure that you get the correct size for the forecasted_value
        forecasted_value_size = forecasted_value.size(2)

        # Reshape the forecasted value based on its size
        forecasted_value = forecasted_value.view(1, 1, forecasted_value_size)

        # Update the input sequence by removing the first element and adding the forecasted value at the end
        input_sequence = torch.cat((input_sequence[:, 1:, :], forecasted_value), dim=1)

# Convert the forecasted value back to the original scale using the scaler
forecasted_value_unscaled = scaler.inverse_transform(forecasted_value.cpu().numpy())

print(f"Forecasted value of 'close' for the next time step: {forecasted_value_unscaled[0][0][0]}")


IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [ ]:

forecasted_value_size = forecasted_value.size(-1)


Forma de forecasted_value: torch.Size([1, 1])


In [ ]:
print(f"Forma de forecasted_value: {forecasted_value.shape}")

Forma de forecasted_value: torch.Size([1, 1])
